In [10]:
import mesa
import numpy as np
import matplotlib.pyplot as plt

## Agent definitions

In [19]:
class EconomicAgent(mesa.Agent):
    'money-seeking agent'
    def __init__(self, id, model):
        super().__init__(id, model)
        
        #agent's attributes:
        # self.wealth = (np.random.pareto(2) + 1) * 10
        self.wealth = 10 #mutable
        self.prosperity = 1 #fixed

        self.criminality = 0 #mutable
        self.criminality_tendency = 0  #fixed
    def step(self):
        print(self.id)
    
# ''' TODO functionalites:
#         Taxation
#         Voting
#         Crimes
#         Policing
#         Probabilistic learning of crime rates
#         Functions for determining optimal actions based on expected utility

# '''

' TODO functionalites:\n        Taxation\n        Voting\n        Crimes\n        Policing\n        Probabilistic learning of crime rates\n        Functions for determining optimal actions based on expected utility\n\n'

## Model definition

In [20]:
class EconomicModel(mesa.Model):
    def _init__(self, N):
        super().__init__()
        self.num_agents = N
        #create scheduler
        self.schedule = mesa.time.RandomActivation(self)

        # create agents
        for i in range(self.num_agents):
            a = EconomicAgent(i, self)
            self.schedule.add(a)
    def step(self):
        self.schedule.step()
    
    


## Running, experiments

In [18]:
starter_model = EconomicModel(10)
starter_model.step()


AttributeError: 'NoneType' object has no attribute 'step'